In [55]:
#=================================================================================================#
# Modeling the data

# Main Metric
#.         Precision
#.         Precision

#=================================================================================================#
import os
import pandas as pd
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score
# from tensorflow.keras.callbacks import ReduceLROnPlateau

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Global configuration
DATA_ROOT=Path("../data/interim/").resolve()
DATASET_DIR=Path.joinpath(DATA_ROOT,'creditcard.csv')


In [56]:
# Load the data
def load_dataset(path_dir):
    df=pd.read_csv(path_dir,sep=",")
    return df

df=load_dataset(DATASET_DIR)
X=df.drop('Class',axis=1)
y=df['Class']
print(f"The shape of X:  {X.shape}")
print(f"The shape of the target:  {X.shape}")

The shape of X:  (284807, 30)
The shape of the target:  (284807, 30)


In [57]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)
print(X_train.shape)

(227845, 30)


In [58]:
# Process the target# 
y.value_counts()


Class
0    284315
1       492
Name: count, dtype: int64

In [59]:
legit=df[df['Class']==0]
fraud=df[df['Class']==1]
print(legit.shape)
print(fraud.shape)



(284315, 31)
(492, 31)


In [70]:
# Create a simple model
def create_simple_model(input_shape,initial_lr=0.001):
    model=tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(input_shape,1)),
            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dense(2,activation="softmax")
        ]
        
    )

    #compile the model
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss="mse",
        metrics=['mae']
        
    )
    return  model
# Create a simple model
def create_fairly_medium_model(input_shape):
    model=tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(input_shape,1)),
            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dense(2,activation="softmax")
        ]
        
    )

    #compile the model
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss="sparse_categorical_crossentropy",
        metrics=['precision']
        
    )
    return  model

    # Create a simple model
def create_advanced_model(input_shape):
    model=tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=(input_shape,1)),
            tf.keras.layers.Dense(128,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(64,activation="relu"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),

            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dense(32,activation="relu"),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(2,activation="softmax")
        ]
        
    )

    #compile the model
    model.compile(
        optimizer= tf.keras.optimizers.Adam(learning_rate=initial_lr),
        loss="mse",
        metrics=['mae']
        
    )
    return  model

# setting callbacks functions
    # def set_callbacks():
    #     callbacks=[
            
    #     ]
    #     return callbacks

In [71]:
# A function to train the model with adaptive lr.
def train_with_lr_plateau():
    # Load and preprocess data
    # X_train, X_test, y_train, y_test, feature_names = load_and_preprocess_data()
    
    # Create model with higher initial learning rate
    initial_lr = 0.01  # Start with higher learning rate
    model = create_simple_model(X_train.shape[1], initial_lr)
    
    # Create checkpoint directory
    checkpoint_dir = 'creditcard_checkpoints'
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    # Define callbacks
    lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Reduce LR by half when plateauing
        patience=10,  # Wait for 10 epochs before reducing LR
        min_delta=0.0001,
        min_lr=1e-6,
        verbose=1
    )
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=25,  # Increased patience to allow LR reduction to take effect
        min_delta=0.0001,
        mode='min',
        restore_best_weights=True,
        verbose=1
    )
    
    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.keras'),
        monitor='val_loss',
        mode='min',
        save_best_only=True,
        verbose=1
    )
    
    
    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=64,
        validation_split=0.2,
        callbacks=[early_stopping, lr_reducer, checkpoint],
        verbose=1
    )
    
    
    return model, history, X_test, y_test

In [72]:
def plot_training_history(history):
    """Plot training curves including learning rate changes"""
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))
    
    # Plot loss
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss (MSE)')
    ax1.legend()
    ax1.grid(True)
    
    # Plot MAE
    ax2.plot(history.history['mae'], label='Training MAE')
    ax2.plot(history.history['val_mae'], label='Validation MAE')
    ax2.set_title('Model Mean Absolute Error')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MAE')
    ax2.legend()
    ax2.grid(True)
    
    # Plot Learning Rate
    ax3.plot(history.history['learning_rate'], label='Learning Rate')
    ax3.set_title('Learning Rate Over Time')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('Learning Rate')
    ax3.set_yscale('log')  # Use log scale for better visualization
    ax3.legend()
    ax3.grid(True)
    
    plt.tight_layout()
    plt.show()

In [73]:
# Results Evaluation function
def evaluate_results(model, X_test, y_test, history):
    """Evaluate and print model results"""
    # Evaluate on test set
    test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
    
    # Get training statistics
    best_epoch = np.argmin(history.history['val_loss']) + 1
    total_epochs = len(history.history['loss'])
    best_val_loss = min(history.history['val_loss'])
    best_val_mae = min(history.history['val_mae'])
    
    # Get learning rate statistics
    initial_lr = history.history['learning_rate'][0]
    final_lr = history.history['learning_rate'][-1]
    num_lr_drops = sum(1 for i in range(1, len(history.history['learning_rate']))
                      if history.history['learning_rate'][i] < history.history['learning_rate'][i-1])
    
    print("\nModel Performance Summary:")
    print("-" * 50)
    print(f"Best validation loss achieved at epoch: {best_epoch}/{total_epochs}")
    print(f"Best validation loss (MSE): {best_val_loss:.4f}")
    print(f"Best validation MAE: {best_val_mae:.4f}")
    print(f"Test loss (MSE): {test_loss:.4f}")
    print(f"Test MAE: {test_mae:.4f}")
    print("\nLearning Rate Summary:")
    print(f"Initial learning rate: {initial_lr:.6f}")
    print(f"Final learning rate: {final_lr:.6f}")
    print(f"Number of learning rate reductions: {num_lr_drops}")
    print("-" * 50)

In [75]:
# ================================================================= #
#          Practice Part: Run the Code for credit card Project
# ================================================================= #
# 1. Train model
model, history, X_test, y_test = train_with_lr_plateau()

Epoch 1/200


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/compile_loss/mse/sub/BroadcastGradientArgs defined at (most recent call last):
  File "C:\ProgramData\miniforge3\envs\dll\Lib\runpy.py", line 198, in _run_module_as_main

  File "C:\ProgramData\miniforge3\envs\dll\Lib\runpy.py", line 88, in _run_code

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\ProgramData\miniforge3\envs\dll\Lib\asyncio\base_events.py", line 608, in run_forever

  File "C:\ProgramData\miniforge3\envs\dll\Lib\asyncio\base_events.py", line 1936, in _